In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

start_time = datetime.datetime.now()

message = input("Enter message: ")
client_socket.sendall(message.encode())

response = client_socket.recv(1024)
print(f"Server response: {response.decode()}")

end_time = datetime.datetime.now()

time_taken = end_time - start_time
print(f"Time taken to send data: {time_taken}")

client_socket.close()

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)  

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")
    data = client_socket.recv(1024)
    print(f"Received: {data.decode()}")
    client_socket.sendall(b"ACK: " + data)
    client_socket.close()

In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

start_time = datetime.datetime.now()

message = input("Enter message: ")
client_socket.sendto(message.encode(), ('localhost', 65432))

response, server_address = client_socket.recvfrom(1024)
print(f"Server response: {response.decode()}")

end_time = datetime.datetime.now()

time_taken = end_time - start_time
print(f"Time taken to send data using UDP: {time_taken}")

client_socket.close()

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65432))

print("UDP Server is listening...")

while True:
    data, client_address = server_socket.recvfrom(1024)
    print(f"Received from {client_address}: {data.decode()}")
    server_socket.sendto(b"ACK: " + data, client_address)

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)  

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")
    
    with open('received_file.txt', 'a') as f:
        data = client_socket.recv(1024)
        print(f"Received: {data.decode()}")
        f.write(data.decode() + "\n")  
    
    client_socket.sendall(b"ACK: " + data)
    client_socket.close()

In [ ]:
import socket

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

with open('file_to_send.txt', 'rb') as f:
    client_socket.sendfile(f)

client_socket.close()

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)  

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")
    
    with open('received_file.txt', 'wb') as f:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            f.write(data)
    
    print("File received!")
    client_socket.close()

In [ ]:
import socket
import threading

def handle_client(client_socket, client_address):
    print(f"Connected to {client_address}")
    while True:
        data = client_socket.recv(1024)
        if not data:
            break
        print(f"Received from {client_address}: {data.decode()}")
        client_socket.sendall(b"ACK: " + data)
    client_socket.close()

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)  

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    client_thread.start()

In [ ]:
import socket
import threading

clients = []

def broadcast(message, sender_socket):
    for client in clients:
        if client != sender_socket:
            client.sendall(message)

def handle_client(client_socket, client_address):
    print(f"Connected to {client_address}")
    clients.append(client_socket)
    while True:
        data = client_socket.recv(1024)
        if not data:
            break
        broadcast(data, client_socket)
    client_socket.close()
    clients.remove(client_socket)

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)  

print("Chat Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    client_thread.start()

In [ ]:
import socket
import threading

def receive_messages(client_socket):
    while True:
        data = client_socket.recv(1024)
        if not data:
            break
        print(f"Received: {data.decode()}")

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

receive_thread = threading.Thread(target=receive_messages, args=(client_socket,))
receive_thread.start()

while True:
    message = input("Enter message: ")
    client_socket.sendall(message.encode())